In [ ]:
#Import Base Model Pretrained
base_model = EfficientNetV2B0(
    include_top=False,
    weights="imagenet",
    input_shape=IMG_SHAPE,
)

In [ ]:
#Ensemble Model using Average Method

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average
model_1 = load_model('path/model.h5')
model_1 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='EfficientNetV2B0')
model_2 = load_model('path/model.h5')
model_2 = Model(inputs=model_2.inputs,
                outputs=model_2.outputs,
                name='MobileNetV3Large')
models = [model_1, model_2]
model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
average_ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

average_ensemble_model.compile(optimizer=Adam(learning_rate=0.0001),
             loss='categorical_crossentropy',
             metrics=["accuracy"])

early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=3)
history = average_ensemble_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stop]
)

In [ ]:
#Ensemble Model Using Concatenation Method

model_MobileNetV3Large = MobileNetV3Large(
    include_top=False,
    alpha=1.0,
    weights="imagenet",
    input_shape=IMG_SHAPE,
)

model_EfficientNetV2B0 = EfficientNetV2B0(
    include_top=False,
    weights="imagenet",
    input_shape=IMG_SHAPE,
)

for layer in model_MobileNetV3Large.layers:
        layer.trainable = False


for layer in model_EfficientNetV2B0.layers:
        layer.trainable = False

models = [model_MobileNetV3Large, model_EfficientNetV2B0]

inp = Input((224,224,3))

mobileNetV3Large_process = Lambda(process_mobileNetV3Large)(inp)
MobileNetV3Large = model_MobileNetV3Large(mobileNetV3Large_process)

efficientNetV2B0_process = Lambda(process_efficientNetV2B0)(inp)
EfficientNetV2B0 = model_EfficientNetV2B0(efficientNetV2B0_process)

x = Concatenate()([MobileNetV3Large, EfficientNetV2B0])
x = Flatten(name="flatten")(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
out = Dense(10, activation='softmax')(x)

concatenation_ensemble_model = Model(inp, out)
concatenation_ensemble_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

class_names = train_ds.class_names
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=3)

history = concatenation_ensemble_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stop]
)